In [1]:
### Lines below will not go inside the function
import numpy as np
import pandas as pd
import patsy
df = pd.read_csv('DebTrivedi.csv',index_col = [0])
sel = np.array([1, 6, 7, 8, 13, 15, 18])-1
df = df.iloc[:,sel]
# produce design matrices from R-style formula
X_formula = 'ofp ~ hosp + health + numchron + gender + school + privins'
y, X = patsy.dmatrices(X_formula, df, return_type='dataframe')
Z_formula = 'ofp ~ health'
Z = patsy.dmatrices(Z_formula, df, return_type='dataframe')[1]

In [2]:
class Logit(object):
    def __init__(self):
        self.linkclass = sm.genmod.families.links.logit
    def link(self, mu):
        return mu/(1.0 + mu)
    def link_inv(self, eta):
        thresh = 30.0
        eta = np.minimum(np.maximum(eta,-thresh), thresh)
        exp_eta = np.exp(eta)
        return exp_eta/(1+exp_eta)
    def link_inv_deriv(self, eta):
        thresh = 30.0
        eta[abs(eta) > thresh] = FLOAT_EPS
        return np.exp(eta)/(1+np.exp(eta))**2

class Probit(object):
    def __init__(self):
        self.linkclass = sm.genmod.families.links.probit
    def link(self, mu):
        return st.norm.ppf(mu)
    def link_inv(self, eta):
        thresh = -st.norm.ppf(FLOAT_EPS)
        eta = np.minimum(np.maximum(eta,-thresh),thresh)
        return st.norm.cdf(eta)
    def link_inv_deriv(self, eta):
        return np.maximum(st.norm.pdf(eta),FLOAT_EPS)
    
class CLogLog(object):
    def __init__(self):
        self.linkclass = sm.genmod.families.links.cloglog
    def link(self, mu):
        return np.log(-np.log(1 - mu))
    def link_inv(self, eta):
        return np.maximum(np.minimum(-np.expm1(-np.exp(eta)),1-FLOAT_EPS),FLOAT_EPS)
    def link_inv_deriv(self, eta):
        eta = np.minimum(eta,700)
        return np.maximum(np.exp(eta)*np.exp(-np.exp(eta)),FLOAT_EPS)
    
class Cauchit(object):
    def __init__(self):
        self.linkclass = sm.genmod.families.links.cauchy
    def link(self, mu):
        return st.cauchy.ppf(mu)
    def link_inv(self, eta):
        thresh = -st.cauchy.ppf(FLOAT_EPS)
        eta = np.minimum(np.maximum(eta,-thresh),thresh)
        return st.cauchy.cdf(eta)
    def link_inv_deriv(self, eta):
        return nnp.maximum(st.cauchy.pdf(eta),FLOAT_EPS)
    
class Log(object):
    def __init__(self):
        self.linkclass = sm.genmod.families.links.log
    def link(self, mu):
        return np.log(mu)
    def link_inv(self, eta):
        return np.maximum(np.exp(eta), FLOAT_EPS)
    def link_inv_deriv(self, eta):
        return np.maximum(np.exp(eta), FLOAT_EPS)

In [3]:
def setLinkClass(argument):
    Link = {
        'logit': Logit(),
        'probit': Probit(),
        'cloglog': CLogLog(),
        'cauchit': Cauchit(),
        'log': Log(),
    }
    return Link.get(argument, Logit)

In [5]:
## Function starts
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy as sp
import scipy.stats as st
import sys
import warnings

FLOAT_EPS = np.finfo(float).eps


## convenience variables
Y = np.squeeze(y.values)
n = len(Y)
kx = X.shape[1] # Number of columns in X
kz = Z.shape[1]
Y0 = Y <= 0
Y1 = Y > 0

## sanity checks
if len(Y) < 1:
    sys.exit("empty model")
if np.all(Y > 0):
    sys.exit("invalid dependent variable, minimum count is not zero")  
if np.array_equal(np.asarray(Y), (np.round(Y + 0.001)).astype(int)) is False:
    sys.exit("invalid dependent variable, non-integer values")
Y = (np.round(Y + 0.001)).astype(int)
if np.any(Y < 0):
    sys.exit("invalid dependent variable, negative counts")
    
weights = offsetx = offsetz = None
control = {'start':None, 'EM':True, 'method':'BFGS', 'tol':None,\
                        'options':None}
## weights and offset

if weights is None:
    weights = 1.0
weights = np.ndarray.flatten(np.array(weights))
if weights.size == 1:
    weights = np.repeat(weights,n)
weights = pd.Series(data = weights, index = X.index)

if offsetx is None:
    offsetx = 0.0
offsetx = np.ndarray.flatten(np.array(offsetx))
if offsetx.size == 1:
    offsetx = np.repeat(offsetx,n)

if offsetz is None:
    offsetz = 0.0
offsetz = np.ndarray.flatten(np.array(offsetz))
if offsetz.size == 1:
    offsetz = np.repeat(offsetz,n)
    
## binary link processing
linkstr = 'logit'
linkList = ['logit','probit','cauchit','cloglog','log']
if linkstr not in linkList:
    sys.exit(linkstr +" link not valid. Available links are: " + str(linkList))
link = setLinkClass(linkstr)
linkobj = link

def ziPoisson(parms, sign = 1.0):
    ## count mean
    mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
    ## binary mean
    phi = link.link_inv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
    ## log-likelihood for y = 0 and y >= 1
    loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
    loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)
    ## collect and return
    loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
    return sign*loglik


def ziNegBin(parms, sign = -1.0):
    ## count mean
    mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
    ## binary mean
    phi = linkobj.link_inv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
    ## negbin size
    theta = np.exp(parms[kx+kz])
    
    ## log-likelihood for y = 0 and y >= 1 sp.stats.poisson.logpmf(Y, mu)
    loglik0 = np.log(phi + np.exp(np.log(1-phi) + \
                                   st.nbinom.logpmf(0,*convert_params(theta = theta, mu = mu)) ) )
    loglik1 = np.log(1-phi) + st.nbinom.logpmf(Y,*convert_params(theta = theta, mu = mu))

    ## collect and return
    loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
    return sign*loglik
  
def ziGeom(parms, sign = 1.0):
    return ziNegBin(np.hstack((parms, 0)), sign)

def gradPoisson(parms, sign = 1.0):
    ## count mean
    eta = np.dot(X,parms[np.arange(kx)]) + offsetx
    mu = np.exp(eta)
    ## binary mean
    etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
    muz = link.link_inv(etaz)
    ## densities at 0
    clogdens0 = -mu
    dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)
    ## working residuals  
    wres_count = np.where(Y1,Y-mu,-np.exp(-np.log(dens0) + 
                                          np.log(1 - muz) + clogdens0 + np.log(mu))) 
    link_etaz = link.link_inv_deriv(etaz)
    wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
    
    
    return sign*(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)

def convert_params(mu, theta):
    """
    Convert mean/dispersion parameterization of a negative binomial to the ones scipy supports

    See https://en.wikipedia.org/wiki/Negative_binomial_distribution#Alternative_formulations
    """
    r = theta
    var = mu + 1 / r * mu ** 2
    p = (var - mu) / var
    return r, 1 - p

def gradGeom(parms, sign = 1.0):
    ## count mean
    eta = np.dot(X,parms[np.arange(kx)]) + offsetx
    mu = np.exp(eta)
    ## binary mean
    etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
    muz = linkobj.link_inv(etaz) 

    ## densities at 0
    clogdens0 = st.nbinom.logpmf(0,*convert_params(theta = 1, mu = mu))
    dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)

    ## working residuals  
    wres_count = np.where(Y1,Y - mu*(Y + 1)/(mu + 1), \
                              -np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0 +\
                                      -np.log(mu+1) + np.log(mu))) 
    link_etaz = linkobj.link_inv_deriv(etaz)
    wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
      
    return sign*(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)

def gradNegBin(parms, sign = 1.0): 
    ## count mean
    eta = np.dot(X,parms[np.arange(kx)]) + offsetx
    mu = np.exp(eta)
    ## binary mean
    etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
    muz = linkobj.link_inv(etaz)    
    ## negbin size
    theta = np.exp(parms[kx+kz])

    ## densities at 0
    clogdens0 = st.nbinom.logpmf(0,*convert_params(theta = theta, mu = mu))
    dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)
        
    ## working residuals  
    wres_count = np.where(Y1,Y - mu*(Y + theta)/(mu + theta), \
                              -np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0 + np.log(theta) +\
                                      -np.log(mu+theta) + np.log(mu))) 
    link_etaz = linkobj.link_inv_deriv(etaz)
    wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
        
    wres_theta = theta*np.where(Y1, sp.special.digamma(Y + theta) - sp.special.digamma(theta) +\
                                   np.log(theta) - np.log(mu + theta) + 1 - (Y + theta)/(mu + theta),\
                                   np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0)*\
                                   (np.log(theta) - np.log(mu + theta) + 1 - theta/(mu+theta) ) )
        
    return sign*(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z, \
                               np.expand_dims(wres_theta,axis=1)))).sum(axis=0)


## Parameters: mention these in class definition
##-----------------------------------------------

reltol =  (np.finfo(float).eps)**(1/1.6)
method = 'L-BFGS-B'
dist = 'negbin'
##-----------------------------------------------
#reltol = control['tol']
if reltol is None:
    reltol =  (np.finfo(float).eps)**(1/1.6)


if dist not in ['Poisson','negbin','geom']:
    sys.exit(dist+" method not yet implemented")
if dist is 'Poisson':
    loglikfun = ziPoisson
    gradfun = gradPoisson
elif dist is 'negbin':
    loglikfun = ziNegBin
    gradfun = gradNegBin
else:
    loglikfun = ziGeom
    gradfun = gradGeom
    
options = control['options']
if options is None:
    options = {'disp': False, 'maxiter': 10000,'ftol': reltol}
start = None

# starting values
if start is not None:
    valid = True
    if ('count' in start) is False:
        valid = False
        warnings.warn("invalid starting values, count model coefficients not specified")
        start['count'] = pd.Series(np.repeat(0,kx), index = X.columns.values)
    if ('zero' in start) is False:
        valid = False
        warnings.warn("invalid starting values, zero model coefficients not specified")
        start['zero'] = pd.Series(np.repeat(0,kz), index = Z.columns.values)
    if(len(start['count']) != kx):
        valid = False
        warning("invalid starting values, wrong number of count model coefficients")
    if(len(start['zero']) != kz):
        valid = False
        warning("invalid starting values, wrong number of zero model coefficients")
    if dist is 'negbin':
        if ('theta' in start) is False:
            start['theta'] = 1.0
        start = {'zero':start['zero'], 'count':start['count'], 'theta' : (start['theta'][0]).astype(float)}
    else:
            start = {'zero':start['zero'], 'count':start['count']}    
    if valid is False:
        start = None

if start is None:
## EM estimation of starting values
    
    model_count = sm.GLM(endog = Y, exog = X, family = sm.families.Poisson(),\
                                  offset = offsetx , freq_weights = weights).fit()
    model_zero = sm.GLM(Y0.astype(int), exog = Z, family=sm.families.Binomial(link = link.linkclass), \
                   offset = offsetz , freq_weights = weights).fit()
    start = {'zero':model_zero.params, 'count':model_count.params}
        
    if (control['EM'] is True) and (dist in ['Poisson']):
        mui = model_count.predict()
        probi = model_zero.predict()
        probi = probi/(probi + (1-probi)*sp.stats.poisson.pmf(0, mui))
        probi[Y1] = 0
        probi
        if dist is 'Poisson':
            ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
        if dist is 'negbin':
            ll_new = ziPoisson(np.hstack((start['count'].values,start['zero'].values)))
        ll_old = 2 * ll_new
    
        while np.absolute((ll_old - ll_new)/ll_old) > reltol :
            ll_old = ll_new
            model_count = sm.GLM(endog = Y, exog = X, family = sm.families.Poisson(),\
                                  offset = offsetx , freq_weights = weights*(1-probi) \
                                              ).fit(sm.families.Poisson().starting_mu\
                                                    (y=start['count'].values)) 
            
            model_zero = sm.GLM(probi, exog = Z, family=sm.families.Binomial(link = link.linkclass),\
                        offset = offsetz, freq_weights = weights \
                               ).fit(sm.families.Binomial().starting_mu(y=start['zero'].values))
                        #,start_params = start['zero']).fit()
            start = {'zero':model_zero.params, 'count':model_count.params}
            if dist is 'negbin':
                start['theta'] = 1.0

            mui = model_count.predict()
            probi = model_zero.predict()
            probi = probi/(probi + (1-probi)*sp.stats.poisson.pmf(0, mui))
            probi[Y1] = 0
            
            if dist is 'Poisson':
                ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
            if dist is 'negbin':
                ll_new = ziPoisson(np.hstack((start['count'].values,start['zero'].values)))
                
    if dist is 'negbin':
        start['theta'] = 1.0
    
    if (control['EM'] is True) and (dist is 'geom'):
        mui = model_count.predict()
        probi = model_zero.predict()
        probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = 1, mu = mui)))
        probi[Y1] = 0
            
        ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
        ll_old = 2 * ll_new  
                           
        while np.absolute((ll_old - ll_new)/ll_old) > reltol :
            ll_old = ll_new
            model_count = sm.GLM(endog = Y, exog = X, family = sm.families.NegativeBinomial(),\
                                  offset = offsetx , freq_weights = weights*(1-probi)\
                                ).fit(start_params =\
                                start['count'].values, scale = 1.0)
            model_zero = sm.GLM(probi, exog = Z, family=sm.families.Binomial(link = linkobj.linkclass),\
                        offset = offsetz, freq_weights = weights).fit(start_params = start['zero'].values)
            start = {'zero':model_zero.params, 'count':model_count.params}

            mui = model_count.predict()
            probi = model_zero.predict()
            probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = 1, mu = mui)))
            probi[Y1] = 0

            ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
                
    #if (control['EM'] is True) and (dist is 'bnegbin'):
        #mui = model_count.predict()
        #probi = model_zero.predict()
        #probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = start['theta'], mu = mui)))
        #probi[Y1] = 0
            
        #ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values,np.log(start['theta']))))
        #ll_old = 2 * ll_new 
        #start['count2'] = start['count']
        #start['count2']['alpha'] = 1.0
        
        #while np.absolute((ll_old - ll_new)/ll_old) > reltol :
        #    ll_old = ll_new
            #temp = np.expand_dims(weights*(1-probi),axis=1)*X

       #     model_count = sm.NegativeBinomial(endog = Y, exog = np.expand_dims(weights*(1-probi),axis=1)*X ).fit(\
       #                                                              start_params = start['count2'],disp=0)
            
        #    model_count2 = sm.GLM(endog = Y, exog = X, family = \
         #                            sm.families.NegativeBinomial(alpha = model_count.params[-1]),\
        #                         offset = offsetx , freq_weights = weights*(1-probi) \
        #                             ).fit(start_params = model_count.params[np.arange(kx)])
        #    
        #    model_zero = sm.GLM(probi, exog = Z, family=sm.families.Binomial(link = linkobj.linkclass),\
        #                offset = offsetz, freq_weights = weights \
        #                ).fit(start_params = start['zero'])
        #    start = {'zero':model_zero.params, 'count':model_count2.params[np.arange(kx)],\
        #             'theta':model_count.params[-1]}
        #    start['count2'] = start['count']
        #    start['count2']['alpha'] = model_count.params[-1]
    
        #    mui = model_count2.predict()
        #    probi = model_zero.predict()
        #    probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = start['theta'], mu = mui)))
        #    probi[Y1] = 0
        #    print(start['zero'])
        #    ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values,np.log(start['theta']))))
            
        #print(start['count'],start['zero'],start['theta'])

## ML Estimation
if (dist is 'negbin'):
    x0 = np.hstack((start['count'].values,start['zero'].values,\
                                         np.log(start['theta'])))
else:
    x0 = np.hstack((start['count'].values,start['zero'].values))

print('x0')
print(x0)

maxiter = 10000
fit = sp.optimize.minimize(loglikfun, args=(-1.0,), x0 = x0,
            method=method, jac=gradfun, options=options)
print(fit.success)
print(fit.x)
#fit = sp.optimize.minimize(loglikfun, args=(-1.0/2,), x0 = np.hstack((start['count'].values,start['zero'].values)),\
#            method=method, jac=gradfun, options=options, tol = reltol)

## coefficients and covariances
coefc = pd.Series(data = fit.x[0:kx], index = X.columns.values)
coefz = pd.Series(data = fit.x[kx:kx+kz], index = Z.columns.values)
print(fit.hess_inv[np.arange(kx+kz)])
#vc = pd.DataFrame(data = -fit.hess_inv, index = np.append(X.columns.values, Z.columns.values),\
#                 columns = np.append(X.columns.values, Z.columns.values))

## fitted and residuals
mu = np.exp(np.dot(X,coefc)+offsetx)
phi = link.link_inv(np.dot(Z,coefz)+offsetz)
Yhat = (1-phi) * mu
res = np.sqrt(weights) * (Y - Yhat)

## effective observations
nobs = np.sum(weights > 0)

print('mu')
print(mu)
print(phi)
print(Yhat)
print(res.head)
print('done')

x0
[ 1.0288742  -0.3619932   0.24830697 -0.11231992  0.20168688  0.16479739
  0.14663928  0.02614299 -1.70004062  0.54223946 -0.4269797   0.        ]
True
[ 0.94183181 -0.33942863  0.32834673 -0.12464251  0.21833937  0.22253464
  0.17370236  0.02672757 -4.94248687  0.21935443  1.51923149  0.2302238 ]


TypeError: only integer scalar arrays can be converted to a scalar index

In [62]:
fit.x[11]

0.2302238017996861

In [259]:
start['count'][0:kx-1]

Intercept              0.925232
health[T.excellent]   -0.341604
health[T.poor]         0.313515
gender[T.male]        -0.126783
privins[T.yes]         0.224835
hosp                   0.220568
numchron               0.176032
dtype: float64

In [272]:
model_count.predict().shape

(4406,)

In [245]:
(np.expand_dims(weights*(1-probi),axis=1)*X).shape

(4406, 8)

In [280]:
zzz =sm.NegativeBinomial(endog = Y, exog = X).fit(disp=0)

In [ ]:
(Intercept)            hosp      healthpoor healthexcellent        numchron      gendermale 
  0.94205766359   0.22249955552   0.32883743539  -0.32944919630   0.17363300721  -0.12465344621 
         school      privinsyes 
  0.02667055784   0.21831871595


In [240]:
np.expand_dims(weights,axis=1)*X

,Intercept,health[T.excellent],health[T.poor],gender[T.male],privins[T.yes],hosp,numchron,school
1,1.0,0.0,0.0,1.0,1.0,1.0,2.0,6.0
2,1.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0
3,1.0,0.0,1.0,0.0,0.0,3.0,4.0,10.0
4,1.0,0.0,1.0,1.0,1.0,1.0,2.0,3.0
5,1.0,0.0,0.0,0.0,1.0,0.0,2.0,6.0
6,1.0,0.0,1.0,0.0,0.0,0.0,5.0,7.0
7,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
9,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
10,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0


In [207]:
start['theta']

array(1.0)

In [211]:
fit

      fun: 12167.119605533411
 hess_inv: <12x12 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.8205135085e-03,  -3.4590051676e-03,  -1.6154901416e-02,
         6.4643783397e-03,   2.7323399842e-03,  -7.0337889430e-03,
         9.2707042965e-04,   3.3169504670e-03,   1.0870363568e-04,
        -1.4406362619e-05,   1.2311223706e-04,  -2.3799151531e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 31
      nit: 22
   status: 0
  success: True
        x: array([  0.9280519863,  -0.3404561013,   0.3324601949,  -0.1261190728,
         0.2219353182,   0.2234295264,   0.1753917223,   0.0268867633,
       -23.6419870554,  10.0030787297,  20.1465152906,   0.2085841889])

In [182]:
sm.GLM(endog = Y, exog = X, family = sm.families.NegativeBinomial(),\
                                  offset = offsetx , freq_weights = weights \
                                      ).estimate_scale(mui)

1.0835766018911714

In [203]:
zzz = sm.GLM(endog = Y, exog = X, family = sm.families.NegativeBinomial(),\
                                  offset = offsetx , freq_weights = weights \
                                      ).fit()

In [210]:
zzz.estimate_scale(zzz.mu)

AttributeError: 'GLMResults' object has no attribute 'estimate_scale'

In [199]:
zzz.fit().predict()

array([  5.7889681902,   5.879909907 ,  17.4671331898, ...,  11.5237203803,
         4.3657025249,   1.579620449 ])

In [ ]:
fit(start_params = start['count'], scale = 1.0)

In [173]:
float(start['theta'])

1.0

In [222]:
model_count2 = sm.NegativeBinomial(endog = Y, exog = X).fit()
model_count2.params[-1]

Optimization terminated successfully.
         Current function value: 2.762268
         Iterations: 23
         Function evaluations: 26
         Gradient evaluations: 26


0.8287729518072201

In [ ]:
 5.7964973398   5.8768941697  17.6984259711
    5.7964969504 5.8768942944  17.6984149602 #bfgs
 5.7964973464 5.8768941803  17.6984260043 #bfgs
 5.7964973398   5.8768941697  17.6984259711 #lbfgsb
[1] "mu"
           1            2            3            4            5            6 
 5.796497340  5.876894170 17.698425969  7.316259799  5.277586475 10.045588333 
[1] "phi"
              1               2               3               4               5               6 
2.690574468e-11 2.690574468e-11 7.400961624e-03 7.400961624e-03 2.690574468e-11 7.400961624e-03 
[1] "Yhat"
           1            2            3            4            5            6 
 5.796497340  5.876894170 17.567440598  7.262112441  5.277586474  9.971241320 
[1] "res"
            1             2             3             4             5             6 
-0.7964973396 -4.8768941696 -4.5674405981  8.7378875589 -2.2775864744  7.0287586803

[1] "coefc"
    (Intercept)            hosp      healthpoor healthexcellent        numchron      gendermale 
  0.92523166755   0.22056800683   0.31351506308  -0.34160422533   0.17603152992  -0.12678306499 
         school      privinsyes 
  0.02688988115   0.22483485695 
[1] "coefz"
    (Intercept)      healthpoor healthexcellent 
  -24.338681296    19.439964442     9.998584332 

## lbfgsb R
[1] "coefc"
    (Intercept)            hosp      healthpoor healthexcellent        numchron      gendermale 
  1.39367089218   0.15912707344   0.25420135853  -0.30775730378   0.10328033968  -0.06485935293 
         school      privinsyes 
  0.01959106898   0.08544135068 
[1] "coefz"
    (Intercept)      healthpoor healthexcellent 
  -1.7336424019   -0.3991551566    0.4748861455

## bfgs R
[1] "coefc"
    (Intercept)            hosp      healthpoor healthexcellent        numchron      gendermale 
  1.39367089333   0.15912707346   0.25420135854  -0.30775730295   0.10328034008  -0.06485935228 
         school      privinsyes 
  0.01959107857   0.08544135136 
[1] "coefz"
    (Intercept)      healthpoor healthexcellent 
  -1.7336424016   -0.3991551566    0.4748861457

In [72]:
np.set_printoptions(precision=10)
print('coefc')
print(coefc)
print(np.array(coefc))
print('coefz')
print(coefz)

coefc
Intercept              1.393672
health[T.excellent]   -0.307754
health[T.poor]         0.254201
gender[T.male]        -0.064859
privins[T.yes]         0.085441
hosp                   0.159127
numchron               0.103280
school                 0.019591
dtype: float64
[ 1.3936719604 -0.3077536592  0.2542013235 -0.064859146   0.0854410736
  0.1591270576  0.103280223   0.0195909989]
coefz
Intercept             -1.733641
health[T.excellent]    0.474905
health[T.poor]        -0.399157
dtype: float64


In [ ]:
[1] "coefc"
    (Intercept)            hosp      healthpoor healthexcellent        numchron      gendermale 
  0.92523166755   0.22056800683   0.31351506308  -0.34160422533   0.17603152992  -0.12678306499 
         school      privinsyes 
  0.02688988115   0.22483485695 
[1] "coefz"
    (Intercept)      healthpoor healthexcellent 
  -24.338681296    19.439964442     9.998584332 

In [12]:
print('coefc')
print(coefc)
print('coefz')
print(coefz)

coefc
Intercept              1.393672
health[T.excellent]   -0.307754
health[T.poor]         0.254201
gender[T.male]        -0.064859
privins[T.yes]         0.085441
hosp                   0.159127
numchron               0.103280
school                 0.019591
dtype: float64
coefz
Intercept             -1.733642
health[T.excellent]    0.474906
health[T.poor]        -0.399156
dtype: float64
